### 1. Загрузка данных 

In [2]:
import pandas as pd
from datetime import date, timedelta 
import urllib.request
from io import open
from io import StringIO
from geopy.distance import distance
import numpy as np

Чтобы не было путаницы в источниках магнитуд возьмем данные NIED (Япония все-таки)

In [3]:
yst = "27.5"
yen = "42.5"
xst = "130"
xen = "144"
author = "NIED"
type_ = "Any"
start_time = date(1995, 1, 1)
end_time = date(2005, 1, 1)

In [4]:
url = "http://isc-mirror.iris.washington.edu/cgi-bin/web-db-v4?request=COMPREHENSIVE&out_format=CATCSV&searchshape=RECT&bot_lat=" + yst + "&top_lat=" + yen + "&left_lon=" + xst + "&right_lon=" + xen + "&ctr_lat=&ctr_lon=&radius=&max_dist_units=deg&srn=&grn=&start_year=" + str(start_time.year) + "&start_month=" + str(start_time.month) + "&start_day=" + str(start_time.day) + "&start_time=00%3A00%3A00&end_year=" + str(end_time.year) + "&end_month=" + str(end_time.month) + "&end_day=" + str(end_time.day) + "&end_time=00%3A00%3A00&min_dep=&max_dep=&null_dep=on&min_mag=" + "&max_mag=&req_mag_type=" + type_ + "&req_mag_agcy=" + author
csv = urllib.request.urlopen(url).read().decode("utf-8") 

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [5]:
start = csv.find('...')
end = csv.find('STOP', start)
csv = csv[start + 5:end]

In [6]:
df = pd.read_csv(StringIO(csv), sep=",", on_bad_lines='skip')

In [7]:
df.head()

,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR .1,TYPE,MAG
0,1004915,ke,ISC,1997-01-04,12:31:52.00,33.8238,131.1687,13.7,,NIED,Mw,3.8
1,1005850,ke,ISC,1997-01-11,05:50:23.79,31.5974,131.7461,40.0,,NIED,Mw,5.4
2,1006351,ke,ISC,1997-01-14,08:48:35.66,33.2363,132.3621,7.9,,NIED,Mw,3.5
3,1006386,ke,ISC,1997-01-14,14:46:53.62,30.5042,131.5143,34.4,,NIED,Mw,4.5
4,1006639,ke,ISC,1997-01-16,06:10:37.40,31.7345,131.7291,33.8,,NIED,Mw,3.9


In [8]:
df = df.rename(columns = {' EVENTID': 'EVENTID', 'AUTHOR   ': 'AUTHOR', 'DATE      ': 'DATE','TIME       ': 'TIME', 'LAT     ':'LAT', 'LON      ': 'LON', 'AUTHOR   .1': 'AUTHOR MAG', 'TYPE  ': 'TYPE MAG', 'MAG ': 'MAG'})

In [9]:
df.to_csv('raw_data.csv', index=False)

### 2. Обработка данных

In [4]:
df = pd.read_csv('raw_data.csv')

In [5]:
df.tail()

,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR MAG,TYPE MAG,MAG
6179,7457443,fe,ISC,2004-12-30,15:15:48.62,33.0934,140.7714,47.3,,NIED,Mw,3.8
6180,7457475,ke,ISC,2004-12-30,17:28:16.61,28.9417,130.5682,40.0,TRUE,NIED,Mw,4.3
6181,7457810,fe,ISC,2004-12-31,08:30:22.60,39.8090,142.4733,56.2,,NIED,Mw,4.3
6182,7457844,ke,ISC,2004-12-31,09:58:50.39,39.8976,143.5029,6.0,TRUE,NIED,Mw,4.2
6183,7458037,fe,ISC,2004-12-31,20:13:48.62,36.8474,140.9306,88.2,,NIED,Mw,5.0


In [6]:
df['COORDINATE'] = df.apply(lambda x: (x['LAT'], x['LON']), axis = 1)

In [7]:
df['DATE'] = df.apply(lambda x: date(int(x['DATE'][:4]), int(x['DATE'][5:7]), int(x['DATE'][8:10])), axis=1)

### 3.1 Подсчет средней магнитуды и плотности землетрясений 

**Первый способ**: для каждой точки найдем те, которые отстоят не больше, чем на $R$, и по времени произошли раньше не больше, чем на $T$ дней (воспользуемся упорядоченностью таблицы)

In [65]:
def density(x, df, R = 100, T = 365):
    suit_r = [distance(x['COORDINATE'], df.iloc[i]['COORDINATE']).km < R for i in range(0, df.shape[0])]
    
    suit_t = [False] * df.shape[0]
    j = x.to_frame().T.index[0]
    while j >= 0:
        if x['DATE'] - df.iloc[j]['DATE'] < timedelta(days = T):
            suit_t[j] = True
            j -= 1
        else:
            break
            
    res = suit_r and suit_t
    n = res.count(True)
    density = n / (np.pi * R ** 2 * T)
    return density


In [85]:
def mean_mag(x, df, R = 200, T = 700):
    suit_r = [distance(x['COORDINATE'], df.iloc[i]['COORDINATE']).km < R for i in range(0, df.shape[0])]
    j = x.to_frame().T.index[0]
    suit_t = [False] * df.shape[0]
    while j >= 0:
        if x['DATE'] - df.iloc[j]['DATE'] < timedelta(days = T):
            suit_t[j] = True
            j -= 1
        else:
            break    
    suit = list(map(int, suit_r and suit_t))
    res = suit * df['MAG']
    a = res.to_list()
    m = res.sum() / (len(a) - a.count(0))
    return m

In [88]:
df['DENSITY'] = df.apply(lambda x: density(x, df), axis = 1)

In [89]:
df['MEAN MAG'] = df.apply(lambda x: mean_mag(x, df), axis = 1)

Считалось долго, поэтому сохраним эту таблицу на всякий пожарный

In [93]:
df.to_csv('data_edit_1.csv', index=False)

### 3.2 Подсчет средней магнитуды и плотности землетрясений

**Второй способ**: разобьем участок на сеточку, считая его плоским, в каждой клеточке посчитаем среднюю магнитуду и плотность, а потом сопоставим их каждой точке в клеточке

In [8]:
from math import ceil

In [9]:
yen = float(yen)
yst = float(yst)
xen = float(xen)
xst = float(xst)

In [12]:
R_earth = 6371
R = 200 
y_mean = (yst + yen) / 2
dy = (R * 360) / (2 * np.pi * R_earth)
dx = (R * 360) / (2 * np.pi * R_earth * np.cos(np.deg2rad(y_mean)))

In [13]:
a_x = xen - xst
a_y = yen - yst

In [19]:
grid = np.zeros((ceil(a_x / dx), ceil(a_y / dy)))

In [20]:
grid

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [24]:
print()

27.5


In [21]:
df.head()

,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR MAG,TYPE MAG,MAG,COORDINATE
0,1004915,ke,ISC,1997-01-04,12:31:52.00,33.8238,131.1687,13.7,,NIED,Mw,3.8,"(33.8238, 131.1687)"
1,1005850,ke,ISC,1997-01-11,05:50:23.79,31.5974,131.7461,40.0,,NIED,Mw,5.4,"(31.5974, 131.7461)"
2,1006351,ke,ISC,1997-01-14,08:48:35.66,33.2363,132.3621,7.9,,NIED,Mw,3.5,"(33.2363, 132.3621)"
3,1006386,ke,ISC,1997-01-14,14:46:53.62,30.5042,131.5143,34.4,,NIED,Mw,4.5,"(30.5042, 131.5143)"
4,1006639,ke,ISC,1997-01-16,06:10:37.40,31.7345,131.7291,33.8,,NIED,Mw,3.9,"(31.7345, 131.7291)"


In [48]:
df[(df['LAT'] - yst < dy) & (df['LON'] - xst < dx)].count()[0]

135

In [49]:
for i in range(0, ceil(a_x / dx)):
    for j in range(0, ceil(a_y / dy)):
        grid[i][j] = df[(df['LAT'] - yst < dy * (j + 1)) & (df['LAT'] - yst > dy * j) &  (df['LON'] - xst > dx * i) &  (df['LON'] - xst < dx * (i + 1))].count()[0]

In [50]:
grid / (np.pi * R ** 2)

array([[1.350e+02, 1.670e+02, 2.410e+02, 4.300e+01, 3.000e+00, 2.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 1.500e+01, 2.300e+01, 9.500e+01, 1.090e+02, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 8.000e+00, 1.370e+02, 1.240e+02, 2.400e+01,
        1.000e+00, 1.000e+00, 0.000e+00],
       [0.000e+00, 9.000e+00, 1.400e+01, 1.950e+02, 1.730e+02, 7.100e+01,
        7.000e+00, 0.000e+00, 1.000e+00],
       [1.100e+01, 5.000e+00, 9.100e+01, 1.662e+03, 5.620e+02, 2.280e+02,
        3.500e+01, 3.700e+01, 1.100e+01],
       [4.100e+01, 2.400e+01, 6.000e+01, 1.190e+02, 3.000e+02, 4.730e+02,
        2.700e+02, 2.580e+02, 1.680e+02],
       [0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 2.000e+00, 1.900e+01,
        8.600e+01, 8.800e+01, 3.300e+01]])